# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/board_of_directors.gml")

In [3]:
print(nx.info(g))

Graph with 854 nodes and 2745 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 854 nodes and 2745 edges
Graph with 854 nodes and 2745 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/board_of_directors-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/board_of_directors-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/board_of_directors-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/board_of_directors-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

854

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[507,
 5,
 371,
 106,
 147,
 163,
 20,
 224,
 234,
 384,
 4,
 25,
 453,
 278,
 118,
 27,
 449,
 216,
 71,
 281,
 178,
 142,
 271,
 182,
 426,
 361,
 407,
 381,
 452,
 52,
 227,
 374,
 235,
 138,
 12,
 705,
 189,
 424,
 23,
 181,
 232,
 454,
 37,
 154,
 110,
 111,
 335,
 175,
 516,
 463,
 30,
 522,
 45,
 249,
 248,
 491,
 201,
 59,
 3,
 13,
 593,
 664,
 455,
 703,
 549,
 372,
 363,
 425,
 362,
 365,
 366,
 367,
 95,
 368,
 364,
 369,
 370,
 255,
 221,
 589,
 387,
 79,
 299,
 298,
 732,
 635,
 26,
 306,
 66,
 354,
 786,
 265,
 400,
 629,
 336,
 434,
 437,
 252,
 220,
 537,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 443,
 486,
 489,
 202,
 451,
 93,
 352,
 392,
 598,
 403,
 324,
 518,
 775,
 679,
 385,
 483,
 604,
 606,
 488,
 420,
 728,
 561,
 44,
 48,
 38,
 39,
 51,
 53,
 33,
 43,
 8,
 149,
 150,
 136,
 137,
 148,
 155,
 153,
 152,
 151,
 89,
 90,
 671,
 465,
 668,
 166,
 107,
 108,
 109,
 112,
 113,
 114,
 115,
 116,
 745,
 547,
 445,
 446,
 447,
 448,
 450,
 179,
 477,
 662,
 53

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{507: 854,
 5: 853,
 371: 852,
 106: 851,
 147: 850,
 163: 849,
 20: 848,
 224: 847,
 234: 846,
 384: 845,
 4: 844,
 25: 843,
 453: 842,
 278: 841,
 118: 840,
 27: 839,
 449: 838,
 216: 837,
 71: 836,
 281: 835,
 178: 834,
 142: 833,
 271: 832,
 182: 831,
 426: 830,
 361: 829,
 407: 828,
 381: 827,
 452: 826,
 52: 825,
 227: 824,
 374: 823,
 235: 822,
 138: 821,
 12: 820,
 705: 819,
 189: 818,
 424: 817,
 23: 816,
 181: 815,
 232: 814,
 454: 813,
 37: 812,
 154: 811,
 110: 810,
 111: 809,
 335: 808,
 175: 807,
 516: 806,
 463: 805,
 30: 804,
 522: 803,
 45: 802,
 249: 801,
 248: 800,
 491: 799,
 201: 798,
 59: 797,
 3: 796,
 13: 795,
 593: 794,
 664: 793,
 455: 792,
 703: 791,
 549: 790,
 372: 789,
 363: 788,
 425: 787,
 362: 786,
 365: 785,
 366: 784,
 367: 783,
 95: 782,
 368: 781,
 364: 780,
 369: 779,
 370: 778,
 255: 777,
 221: 776,
 589: 775,
 387: 774,
 79: 773,
 299: 772,
 298: 771,
 732: 770,
 635: 769,
 26: 768,
 306: 767,
 66: 766,
 354: 765,
 786: 764,
 265: 763,
 400: 762,

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/Misc-BoardOfDirectors/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Misc-BoardOfDirectors/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True